In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import hamming_loss, jaccard_score, classification_report

In [2]:
os.chdir("/content/drive/MyDrive/Colab Notebooks/Datasets/")

In [3]:
df = pd.read_csv('maintenance_dataset.csv')
df.head()

,ID,Maintenance_Date,Type_of_Maintenance,Fault_Description,Maintenance_Crew,Duration,Solution
0,1,2020-01-01,Bulb Replacement,Bulb broken,Maintenance Team Gamma,46,Replaced bulb
1,2,2020-01-04,Wiring Repair,Wire fault,Maintenance Team Beta,40,Repaired wiring
2,3,2020-01-08,Fixture Adjustment,Fixture position change,Maintenance Team Beta,23,Adjusted fixture
3,4,2020-01-11,Bulb Replacement,Bulb broken,Maintenance Team Gamma,39,Replaced bulb
4,5,2020-01-15,Wiring Repair,Wire fault,Maintenance Team Gamma,44,Repaired wiring


In [4]:
df1 = pd.get_dummies(df.Fault_Description)
df1.head()

,Bulb broken,Bulb fault,Fixture position change,Wire fault
0,True,False,False,False
1,False,False,False,True
2,False,False,True,False
3,True,False,False,False
4,False,False,False,True


In [5]:
X = df1
X.head(3)

,Bulb broken,Bulb fault,Fixture position change,Wire fault
0,True,False,False,False
1,False,False,False,True
2,False,False,True,False


In [6]:
y = df.Solution
y.head()

0       Replaced bulb
1     Repaired wiring
2    Adjusted fixture
3       Replaced bulb
4     Repaired wiring
Name: Solution, dtype: object

In [7]:
from sklearn.preprocessing import LabelEncoder
encoder= LabelEncoder()
y=encoder.fit_transform(y)
y_list = dict(zip(list(y), df['Solution'].to_list()))

In [8]:
print(y_list)

{2: 'Replaced bulb', 1: 'Repaired wiring', 0: 'Adjusted fixture'}


In [9]:
list(encoder.classes_)

['Adjusted fixture', 'Repaired wiring', 'Replaced bulb']

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)

In [11]:
from sklearn.tree import DecisionTreeRegressor
lr_clf = DecisionTreeRegressor()
lr_clf.fit(X_train,y_train)
lr_clf.score(X_test,y_test)

1.0

In [18]:
def predict_soln(names):
    loc_index = np.where(X.columns==names)[0][0]

    x = np.zeros(len(X.columns))
    if loc_index >= 0:
        x[loc_index] = 1

    return lr_clf.predict([x])[0]

In [19]:
predict_soln('Bulb broken')

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


2.0

In [22]:
print(y_list[predict_soln('Wire fault')])

Repaired wiring


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


In [15]:
import pickle
with open('fault_model.pickle','wb') as f:
    pickle.dump(lr_clf,f)

In [16]:
import json
columns = {
    'data_columns' : [col.lower() for col in X.columns]
}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))

In [17]:
import json
columns = {
    'solution_columns' : [col for col in encoder.classes_]
}
with open("solution_columns.json","w") as f:
    f.write(json.dumps(columns))